# MSABA-02: A/B Testing & Experiment Design — Answer Key

In [ ]:
import pandas as pd, numpy as np
from math import sqrt
import matplotlib.pyplot as plt

df = pd.read_csv("../data/ab_events.csv")
conv = df.groupby("group")["converted"].mean().rename("rate")
n = df.groupby("group").size().rename("n")
print(pd.concat([conv, n], axis=1))

# Two-proportion z-test
p1, p2 = conv["control"], conv["treatment"]
n1, n2 = n["control"], n["treatment"]
p = (p1*n1 + p2*n2)/(n1+n2)
se = sqrt(p*(1-p)*(1/n1 + 1/n2))
z = (p2 - p1) / se
print("Uplift:", (p2-p1).round(4), "  z-score:", round(z,2))

# CI for difference
from scipy.stats import norm
zcrit = norm.ppf(0.975)
ci_low = (p2-p1) - zcrit*se
ci_high = (p2-p1) + zcrit*se
print("95% CI:", round(ci_low,4), "to", round(ci_high,4))

# Power approximation (simple)
# target uplift 1.5pp, alpha .05, power .8
d = 0.015
pbar = 0.06
se0 = sqrt(2*pbar*(1-pbar))
n_per = int(((zcrit + norm.ppf(0.8))**2 * se0**2) / d**2)
print("Approx sample per group for 1.5pp uplift:", n_per)

df.groupby(["group","converted"]).size().unstack(fill_value=0).plot(kind="bar", stacked=True)
plt.title("Conversions by Group")
plt.show()
